# Classification on `emnist`

## 1. Create `Readme.md` to document your work

Explain your choices, process, and outcomes.

## 2. Classify ~~all symbols~~ letters a -> g

### Subset the data

Select only the lowercase letters (a, b, ..., g) for classification

### Choose a model

Your choice of model! Choose wisely...

### Train away!

Is do you need to tune any parameters? Is the model expecting data in a different format?

### Evaluate the model

Evaluate the models on the test set, analyze the confusion matrix to see where the model performs well and where it struggles.

### Investigate subsets

On which classes does the model perform well? Poorly? Evaluate again, excluding easily confused symbols (such as 'O' and '0').

### Improve performance

Brainstorm for improving the performance. This could include trying different architectures, adding more layers, changing the loss function, or using data augmentation techniques.

## 3. Model showdown: upper vs lowercase on abcXYZ

### Subset the data

Select out the set of upper- and lowercase (a, b, c, x, y z, A, B, C, X, Y, Z). Note that some of these classes can be confusing (e.g., x and y).

### Train and tune models

Perform a full model training and hyperparameter tuning.

1. Select candidate models, hyperparameter options, and evaluation metric
2. Set aside a validation hold-out dataset
3. Train models over K splits (use k-fold or train/test split)
    1. Split train using k-fold with the number of folds equal to the number of parameter combinations
    2. Train on k-fold split
    3. Record performance of each set of parameters
    4. Use winning set of parameters to train model on full training set
    5. Record each model's performance on that split's test set
4. Evaluate model performance and promote one model as the winner
5. Train winning model on both train + test
6. Check model performance on the validation hold-out


## 4. (_Optional_) Model comparison: classify even vs odd

**NOTE:** This is a larger dataset (~400k rows) so it will require more memory and time to train models on it. 

Alternatively, you can train models on smaller subsets of the data to get a feel for which models perform better than others. Then train the winning model on the full dataset and validate against the hold-out.

### Subset the data

Select only digits and add a column for 'is_even'. Be sure to create a validation hold-out dataset for later.

### Build and compare models

Train at least two different models, compare the results and choose a winner based on an evaluation metric of your choice.

In [1]:
%pip install -q emnist pandas pyarrow numpy matplotlib seaborn scikit-learn xgboost tensorflow
%reset -f


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import packages
import os
import string
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import emnist
from IPython.display import display, Markdown



# ML packages
# Model selection
from sklearn.model_selection import cross_val_score, cross_validate, KFold, ParameterGrid, train_test_split
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
# XGBoost (SVM)
from xgboost import XGBClassifier
# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import CategoricalCrossentropy
# Keras formatting helper
from scikeras.wrappers import KerasClassifier

# Constants
SIZE = 28
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # or 3 to suppress all warnings

In [3]:
# Define helper functions
def int_to_char(label):
    """Convert an integer label to the corresponding uppercase character."""
    if label < 10:
        return str(label)
    elif label < 36:
        return chr(label - 10 + ord('A'))
    else:
        return chr(label - 36 + ord('a'))

def show_image(row):
    """Display a single image and its corresponding label."""
    image = row['image']
    label = row['label']
    plt.imshow(image, cmap='gray')
    plt.title('Label: ' + int_to_char(label))
    plt.axis('off')
    plt.show()

def show_grid(data, title=None, num_cols=5, figsize=(20, 10)):
    """
    Display a list of images as a grid of num_cols columns.
    images: a list of images, each represented as a 28x28 numpy array
    labels: a list of labels, one for each image
    title: (optional) a title for the plot
    num_cols: (optional) number of columns to use in the grid
    figsize: (optional) size of the figure
    """
    num_images = len(data)
    num_rows = (num_images - 1) // num_cols + 1
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    if title is not None:
        fig.suptitle(title, fontsize=16)
    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            if index < num_images:
                axes[i, j].imshow(data.iloc[index]['image'], cmap='gray')
                axes[i, j].axis('off')
                label = int_to_char(data.iloc[index]['label'])
                axes[i, j].set_title(label)
    plt.show()

# Get a random image of a given label from the dataset
def get_image_by_label(data, label):
    """Get a random image of a given label from the dataset."""
    images = data[data['label'] == label]['image'].tolist()
    return random.choice(images)

# Plot the training and validation accuracy during the training of a model
def plot_accuracy(history):
    """Plot the training and validation accuracy during the training of a model."""
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Plot the training and validation loss during the training of a model
def plot_loss(history):
    """Plot the training and validation loss during the training of a model."""
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Normalize the pixel values of the images in the dataset to have zero mean and unit variance
# This is a common preprocessing step for neural networks, but may not be necessary in all cases
def normalize_images(images):
    """Normalize the pixel values of the images in the dataset to have zero mean and unit variance."""
    images = np.array(images)
    mean = images.mean()
    std = images.std()
    images = (images - mean) / std
    return images.tolist()

# Display metrics for a model
def display_metrics(task, model_name, metrics_dict):
    """Display performance metrics and confusion matrix for a model."""
    metrics_df = pd.DataFrame()
    cm_df = pd.DataFrame()
    for key, value in metrics_dict[task][model_name].items():
        if type(value) == np.ndarray:
            cm_df = pd.DataFrame(value, index=['actual 0', 'actual 1'], columns=['predicted 0', 'predicted 1'])
        else:
            metrics_df[key] = [value]
    display(Markdown(f'# Performance Metrics: {model_name}'))
    display(metrics_df)
    display(Markdown(f'# Confusion Matrix: {model_name}'))
    display(cm_df)

In [4]:
# Load data

# Extract the training split as images and labels
image, label = emnist.extract_training_samples('byclass')

# Add columns for each pixel value (28x28 = 784 columns)
emnist_train = pd.DataFrame()

# Add a column with the image data as a 28x28 array
emnist_train['image'] = list(image)
emnist_train['image_flat'] = emnist_train['image'].apply(lambda x: np.array(x).reshape(-1))

# Add a column showing the label
emnist_train['label'] = label

# Convert labels to characters
class_label = np.array([int_to_char(l) for l in label])

# Add a column with the character corresponding to the label
emnist_train['class'] = class_label

# Repeat for the test split
image, label = emnist.extract_test_samples('byclass')
class_label = np.array([int_to_char(l) for l in label])
emnist_test = pd.DataFrame()
emnist_test['image'] = list(image)
emnist_test['image_flat'] = emnist_test['image'].apply(lambda x: np.array(x).reshape(-1))
emnist_test['label'] = label
emnist_test['class'] = class_label

# Combine the training and test splits
emnist_all = pd.concat([emnist_train, emnist_test], axis=0)

# Subset for only digits 0-9
digits = emnist_all[emnist_all['label'] < 10]

# Subset for lowercase letters
lowercase = emnist_all[(emnist_all['class'] >= 'a') & (emnist_all['class'] <= 'z')]
uppercase = emnist_all[(emnist_all['class'] >= 'A') & (emnist_all['class'] <= 'Z')]

# Subset for upper- and lowercase letters a, b, c, d, e, f, g
a2g = emnist_all[(emnist_all['class'].isin(['a', 'b', 'c', 'd', 'e', 'f', 'g']))]

# Subset for upper- and lowercase letters a, b, c, x, y, z
abcxyz = emnist_all[(emnist_all['class'].isin(['a', 'b', 'c', 'A', 'B', 'C', \
                                               'x', 'y', 'z', 'X', 'Y', 'Z']))]

In [5]:
# Display the size of a2g, abcxyz, digits, and the full dataset
display(Markdown(f'# Dataset Sizes'))
display(Markdown(f'**a2g**: {len(a2g)}'))
display(Markdown(f'**abcxyz**: {len(abcxyz)}'))
display(Markdown(f'**digits**: {len(digits)}'))
display(Markdown(f'**emnist_all**: {len(emnist_all)}'))

# Dataset Sizes

**a2g**: 68795

**abcxyz**: 65926

**digits**: 402953

**emnist_all**: 814255

In [6]:
# FIXME: Classify lettters as uppercase/lowercase
abcxyz['is_upper'] = abcxyz['class'].apply(lambda x: 1 if x.isupper() else 0)

# FIXME: Classify digits as even/odd
even_num = [x for x in range (10) if x%2 == 0]
digits['is_even'] = digits['label'].apply(lambda x: 1 if (x%2 == 0) else 0)

/var/folders/z9/93thyyl57vd2c58_d39dsb7m0000gn/T/ipykernel_74606/3892340535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abcxyz['is_upper'] = abcxyz['class'].apply(lambda x: 1 if x.isupper() else 0)
/var/folders/z9/93thyyl57vd2c58_d39dsb7m0000gn/T/ipykernel_74606/3892340535.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  digits['is_even'] = digits['label'].apply(lambda x: 1 if (x%2 == 0) else 0)


## Task 1: Classify all symbols done in exercise 4 
Follow link in [readme.md](https://github.com/10331333/datasci_223/blob/homework/exercises/4-classification/readme.md "exercise4 readme.md") or download [exercise4.ipynb](https://github.com/10331333/datasci_223/blob/homework/exercises/4-classification/exercise4.ipynb "exercise4.ipynb").



## Task 2: Model showdown: upper vs lowercase on abcXYZ

### Subset the data

Select out the set of upper- and lowercase (a, b, c, x, y z, A, B, C, X, Y, Z). Note that some of these classes can be confusing (e.g., x and y).

### Train and tune models

Perform a full model training and hyperparameter tuning.

1. Select candidate models, hyperparameter options, and evaluation metric
2. Set aside a validation hold-out dataset
3. Train models over K splits (use k-fold or train/test split)
    1. Split train using k-fold with the number of folds equal to the number of parameter combinations
    2. Train on k-fold split
    3. Record performance of each set of parameters
    4. Use winning set of parameters to train model on full training set
    5. Record each model's performance on that split's test set
4. Evaluate model performance and promote one model as the winner
5. Train winning model on both train + test
6. Check model performance on the validation hold-out

In [7]:
# Candidate models are: random forest, xgboost, nn, lr
# Define hyperparameter options: use GridSearchCV() to search for best set of hyperparameters in each model
logireg_param_grid = {'penalty': ['l1', 'l2'],'C': [0.1, 10], 'max_iter': [100, 1000]}
rf_param_grid = {'n_estimators': [50,100], 'max_depth': [None, 10], 'min_samples_split': [2, 5]}
xgb_param_grid = {'n_estimators': [50,100], 'max_depth': [3, 5],'learning_rate': [0.01, 0.1]}
nn_param_grid = {'batch_size': [16, 32, 64, 128],'epochs': [5, 10]}

# Hyperparameter evaluation will be done by accuracy. 
scoring_metric = 'accuracy'

In [8]:
# Set aside validation hold out set = X_val, y_val
scaler = StandardScaler()
X=scaler.fit_transform(abcxyz['image_flat'].tolist()) #scale image 
y=abcxyz['is_upper'].tolist()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)


In [9]:
# Initialize neural network model
def build_nn_model(): 
    """Define model to build neural network model """
    model = Sequential([
        keras.layers.InputLayer(input_shape=(784,)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model 


In [46]:
from itertools import product
""" Define dictionary that list of models, parameters, number of hyperparamter combination to be tested 
    key = 'model_name'
    values = (model, parameter_grid, length_of_parameter_grid) tuple  """

uppercase_models = {
    'logistic_regression': (
        LogisticRegression(random_state=42, solver='liblinear', max_iter=1000), 
        logireg_param_grid, 
        len(list(product(*logireg_param_grid.values())))
    )


}

        'xgboost': (
        XGBClassifier(random_state=42), 
        xgb_param_grid, 
        len(list(product(*xgb_param_grid.values())))
    ),
    'random_forest': (
        RandomForestClassifier(random_state=42), 
        rf_param_grid, 
        len(list(product(*rf_param_grid.values())))
    ),
    'neural_network': (
        KerasClassifier(build_fn=build_nn_model), 
        nn_param_grid, 
        len(list(product(*nn_param_grid.values())))
        )

In [11]:
# Subset for test runs
random.seed(42)
subset_true = random.sample(range(len(X_train)), round(len(X_train)/10))
temp_x = X_train[subset_true,]
temp_y = [y_train[i] for i in subset_true]
len(temp_x)

4615

In [47]:
from sklearn.model_selection import GridSearchCV

# Initialize best_score, best_model, outer_scores
best_score = 0
best_model = None
outer_scores = {'logistic_regression': []}


# Comment depending on training sample subset
# X_subset, y_subset = X_train, y_train
X_subset, y_subset = temp_x, temp_y

# Variables for CV loop

    # Initialize outer cross-validation
outer_fold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Set n_jobs = 1 when executing neural network to avoid pickling issues related to parallelization, 
    # Set n_jobs = -1 for other models to save time
n_jobs = (1 if model_name == 'neural_network' else -1)


""" For each model type, grid search (inner loop) over param_grid and find best hyperparameters. 
    Then cross validate (outer loop) different model types to find the one with best performance. 

    Inner loop:     grid search over hyperparameters, number of fold = combination of hyperparameters 
    Outer loop:     number of fold = number of model types 
    """

# Outer CV loop
for model_name, (model, param_grid, grid_length) in uppercase_models.items():
    # Display step 
    display(Markdown(f'## Now initializing {model_name}'))

    fold_scores = []    

        # Execute the outer cross-validation
    for train_index, test_index in outer_fold.split(X_subset): 
        X_fold_train, X_fold_test = [X_subset[i] for i in train_index], [X_subset[i] for i in test_index]
        y_fold_train, y_fold_test = [y_subset[i] for i in train_index], [y_subset[i] for i in test_index]

        # Initialize inner cross-validation
        inner_fold = KFold(n_splits=grid_length, shuffle=True, random_state=42)
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_fold, scoring='accuracy', verbose=1, n_jobs=n_jobs, refit=True)
            
        # Perform hyperparameter tuning
        grid_search.fit(X_fold_train, y_fold_train)
            
        # 3.C. Record performance of each set of parameters
        best_model = grid_search.best_estimator_
            
        # Evaluate the best model on the outer fold
        y_pred = best_model.predict(X_fold_test)
        accuracy = accuracy_score(y_fold_test, y_pred)
        outer_scores[model_name].append(accuracy)
    

## Now initializing logistic_regression

Fitting 8 folds for each of 8 candidates, totalling 64 fits
Fitting 8 folds for each of 8 candidates, totalling 64 fits
Fitting 8 folds for each of 8 candidates, totalling 64 fits
Fitting 8 folds for each of 8 candidates, totalling 64 fits
Fitting 8 folds for each of 8 candidates, totalling 64 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:


# Model comparison
for model_name, scores in outer_scores.items():
    avg_accuracy = np.mean(scores)
    print(f'{model_name}: Average Accuracy = {avg_accuracy}')


#### 4. Evaluate model performance and promote one model as the winner

In [ ]:
display(Markdown(f'## Best model parameters: {best_model}'))
display(Markdown(f'## Best model average cross-validation score: {best_score}'))

#### 5. Training winning model on both train + test

In [ ]:
# Select best model for training
best_model_instance = best_model[0](**best_model[1])
# Fit the best model on the whole training set
best_model_instance.fit(X_train, y_train)

#### 6. Check model performance on validation hold-out

In [ ]:

y_val_pred = best_model_instance.predict(X_val)
validation_score = accuracy_score(y_val, y_val_pred)
print(f"Validation Hold-out Score: {validation_score}")